In [1]:
import os
import cv2
from shutil import move
import pandas as pd
from glob import glob
from functools import reduce

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Step 1: Retrieve paths of each file in the label directory.

LABEL_FOLDER_PATH = "/home/fox/CodingArea/ObjectDetection/Alpaca_Detection_project/Data/Label" 
IMAGES_FOLDER_PATH = "/home/fox/CodingArea/ObjectDetection/Alpaca_Detection_project/Data/images" 

label_files = os.listdir(LABEL_FOLDER_PATH)
label_files[:10]

['c3e8c57b4cc8d3f5.txt',
 'c328177be6d16b23.txt',
 '164868238bdae200.txt',
 'f8495ecb05e1eb84.txt',
 '7dfbcf3d2e6b88c6.txt',
 '10d6559b8c9fc4e4.txt',
 'bbaa9f767727f442.txt',
 '0e6ba4d54d478f76.txt',
 'a2b6c61ea43efc0a.txt',
 '54e494a15c2999b5.txt']

In [3]:
def extract_info_from_file(file_name, folder_path):
    """Extracts information from a given file within a specified folder.
    
    Args:
        file_name (str): Name of the file to extract information from.
        folder_path (str): Path of the folder containing the file.
        
    Returns:
        list: List of lists containing information extracted from the file.
    """
    
    file_path = os.path.join(folder_path, file_name)
    extracted_data = []

    with open(file_path, "r") as file:
        for line in file:
            cleaned_line = line.strip("\n").split()
            cleaned_line.insert(0, file_name.split('.')[0])
            extracted_data.append(cleaned_line)
            
    return extracted_data

In [4]:

# Extract information from each file
all_data = list(map(lambda file: extract_info_from_file(file, LABEL_FOLDER_PATH), label_files))

# Flatten the nested lists
flattened_data = reduce(lambda x, y: x + y, all_data)

flattened_data

[['c3e8c57b4cc8d3f5', 'Alpaca', '7.039272', '55.04', '357.080068', '471.04'],
 ['c3e8c57b4cc8d3f5',
  'Alpaca',
  '169.58083',
  '270.08',
  '636.7288619999999',
  '621.44'],
 ['c3e8c57b4cc8d3f5', 'Alpaca', '266.209988', '87.68', '581.055142', '359.04'],
 ['c328177be6d16b23',
  'Alpaca',
  '130.743296',
  '112.488908',
  '1023.085568',
  '1005.0855459999999'],
 ['164868238bdae200',
  'Alpaca',
  '64.00023999999999',
  '92.99968',
  '678.99972',
  '1022.000128'],
 ['164868238bdae200',
  'Alpaca',
  '416.0002',
  '106.999808',
  '678.99972',
  '645.999616'],
 ['f8495ecb05e1eb84', 'Alpaca', '0.0', '150.399744', '637.44', '767.360256'],
 ['f8495ecb05e1eb84', 'Alpaca', '567.04', '203.52', '1023.36', '767.360256'],
 ['7dfbcf3d2e6b88c6',
  'Alpaca',
  '113.28',
  '142.022232',
  '589.44',
  '669.1695119999999'],
 ['7dfbcf3d2e6b88c6',
  'Alpaca',
  '321.92',
  '397.91901599999994',
  '481.28',
  '520.749504'],
 ['7dfbcf3d2e6b88c6',
  'Alpaca',
  '503.04',
  '379.36641599999996',
  '927.36',
  

In [5]:
data_columns = ['filename', 'class_name', 'xmin', 'xmax', 'ymin', 'ymax']
df = pd.DataFrame(data=flattened_data, columns=data_columns)

# Calculate and display the number of objects in images
num_objects = df.shape[0]
print(f'Number of objects in images = {num_objects}\n')

# Display the first few rows of the DataFrame
df.head()

Number of objects in images = 829



,filename,class_name,xmin,xmax,ymin,ymax
0,c3e8c57b4cc8d3f5,Alpaca,7.039272,55.04,357.080068,471.04
1,c3e8c57b4cc8d3f5,Alpaca,169.58083,270.08,636.7288619999999,621.44
2,c3e8c57b4cc8d3f5,Alpaca,266.209988,87.68,581.055142,359.04
3,c328177be6d16b23,Alpaca,130.743296,112.488908,1023.085568,1005.0855459999999
4,164868238bdae200,Alpaca,64.00023999999999,92.99968,678.99972,1022.000128


In [6]:
# Display information about the DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829 entries, 0 to 828
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   filename    829 non-null    object
 1   class_name  829 non-null    object
 2   xmin        829 non-null    object
 3   xmax        829 non-null    object
 4   ymin        829 non-null    object
 5   ymax        829 non-null    object
dtypes: object(6)
memory usage: 39.0+ KB


In [7]:
data_columns = ['filename', 'class_name', 'xmin', 'xmax', 'ymin', 'ymax']
df = pd.DataFrame(data=flattened_data, columns=data_columns)

# Convert specific columns to numeric (float) type, which allows for decimal values
numeric_cols = ['xmin', 'xmax', 'ymin', 'ymax']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Convert numeric columns to integer type (if desired)
df[numeric_cols] = df[numeric_cols].astype(int)

# Display updated information about the DataFrame
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829 entries, 0 to 828
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   filename    829 non-null    object
 1   class_name  829 non-null    object
 2   xmin        829 non-null    int64 
 3   xmax        829 non-null    int64 
 4   ymin        829 non-null    int64 
 5   ymax        829 non-null    int64 
dtypes: int64(4), object(2)
memory usage: 39.0+ KB


## Conversion

**For Conversion We Need Width and Height**

In [8]:
def extract_image_info(df, images_folder_path):
    """Extracts information (filename, height, width) from images in a DataFrame.

    Args:
        df (DataFrame): DataFrame containing filenames.
        images_folder_path (str): Path of the folder containing the images.

    Returns:
        list: List of lists containing image information [filename, height, width].
    """

    img_info = []

    for filename in df['filename']:
        img_path = os.path.join(images_folder_path, filename + '.jpg')
        img = cv2.imread(img_path)

        if img is not None:
            img_info.append([filename, img.shape[0], img.shape[1]])
        else:
            print(f"Image '{filename}' could not be read.")

    return img_info

image_information = extract_image_info(df=df ,images_folder_path=IMAGES_FOLDER_PATH)
image_information[:10]

[['c3e8c57b4cc8d3f5', 1024, 718],
 ['c3e8c57b4cc8d3f5', 1024, 718],
 ['c3e8c57b4cc8d3f5', 1024, 718],
 ['c328177be6d16b23', 1006, 1024],
 ['164868238bdae200', 1024, 680],
 ['164868238bdae200', 1024, 680],
 ['f8495ecb05e1eb84', 768, 1024],
 ['f8495ecb05e1eb84', 768, 1024],
 ['7dfbcf3d2e6b88c6', 792, 1024],
 ['7dfbcf3d2e6b88c6', 792, 1024]]

In [9]:
column_names = ["img_name", "width", "height"]
df2 = pd.DataFrame(data=image_information, columns=column_names)
df[['width', 'height']] = df2[['width', 'height']]
df.head()

,filename,class_name,xmin,xmax,ymin,ymax,width,height
0,c3e8c57b4cc8d3f5,Alpaca,7,55,357,471,1024,718
1,c3e8c57b4cc8d3f5,Alpaca,169,270,636,621,1024,718
2,c3e8c57b4cc8d3f5,Alpaca,266,87,581,359,1024,718
3,c328177be6d16b23,Alpaca,130,112,1023,1005,1006,1024
4,164868238bdae200,Alpaca,64,92,678,1022,1024,680


In [11]:
# Calculating normalized center coordinates
df['center_x'] = ((df['xmax'] + df['xmin']) / 2) / df['width']
df['center_y'] = ((df['ymax'] + df['ymin']) / 2) / df['height']

# Calculating normalized width and height
df['w'] = (df['xmax'] - df['xmin']) / df['width']
df['h'] = (df['ymax'] - df['ymin']) / df['height']

df.head()

,filename,class_name,xmin,xmax,ymin,ymax,width,height,center_x,center_y,w,h
0,c3e8c57b4cc8d3f5,Alpaca,7,55,357,471,1024,718,0.030273,0.576602,0.046875,0.158774
1,c3e8c57b4cc8d3f5,Alpaca,169,270,636,621,1024,718,0.214355,0.875348,0.098633,-0.020891
2,c3e8c57b4cc8d3f5,Alpaca,266,87,581,359,1024,718,0.172363,0.654596,-0.174805,-0.309192
3,c328177be6d16b23,Alpaca,130,112,1023,1005,1006,1024,0.120278,0.990234,-0.017893,-0.017578
4,164868238bdae200,Alpaca,64,92,678,1022,1024,680,0.076172,1.250000,0.027344,0.505882


## Assign ID number to object names

In [12]:
# label encoding
def label_encoding(x):
    labels = {'Alpaca':0}
    return labels[x]

df['id'] = df['class_name'].apply(label_encoding)
df.head()

,filename,class_name,xmin,xmax,ymin,ymax,width,height,center_x,center_y,w,h,id
0,c3e8c57b4cc8d3f5,Alpaca,7,55,357,471,1024,718,0.030273,0.576602,0.046875,0.158774,0
1,c3e8c57b4cc8d3f5,Alpaca,169,270,636,621,1024,718,0.214355,0.875348,0.098633,-0.020891,0
2,c3e8c57b4cc8d3f5,Alpaca,266,87,581,359,1024,718,0.172363,0.654596,-0.174805,-0.309192,0
3,c328177be6d16b23,Alpaca,130,112,1023,1005,1006,1024,0.120278,0.990234,-0.017893,-0.017578,0
4,164868238bdae200,Alpaca,64,92,678,1022,1024,680,0.076172,1.250000,0.027344,0.505882,0


## Save The conversion txt

In [13]:
cols = ['filename','id','center_x','center_y', 'w', 'h']
groupby_obj = df[cols].groupby('filename')

In [14]:
groupby_obj.get_group('c3e8c57b4cc8d3f5')

,filename,id,center_x,center_y,w,h
0,c3e8c57b4cc8d3f5,0,0.030273,0.576602,0.046875,0.158774
1,c3e8c57b4cc8d3f5,0,0.214355,0.875348,0.098633,-0.020891
2,c3e8c57b4cc8d3f5,0,0.172363,0.654596,-0.174805,-0.309192


In [17]:
def save_data(filename, folder_path, group_obj):

    
    # save the labels
    text_filename = os.path.join(folder_path, filename+'.txt')
    group_obj.get_group(filename).set_index('filename').to_csv(text_filename,sep=' ',index=False,header=False)
    

In [18]:
folder_path = '/home/fox/CodingArea/ObjectDetection/Alpaca_Detection_project/Data/label'
filename_series = pd.Series(groupby_obj.groups.keys())
filename_series.apply(save_data,args=(folder_path,groupby_obj))

0      None
1      None
2      None
3      None
4      None
       ... 
447    None
448    None
449    None
450    None
451    None
Length: 452, dtype: object